In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

from dotenv import load_dotenv

load_dotenv()

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

In [2]:
t = PromptTemplate(
    template="What is the capital of {country}?",
    input_variables=["country"],
)

t.format(country="France")

'What is the capital of France?'

In [3]:
t = PromptTemplate.from_template("What is the capital of {country}?")

t.format(country="France")

'What is the capital of France?'

In [6]:
from langchain.prompts import ChatPromptTemplate

t = ChatPromptTemplate.from_messages(["What is the capital of {country}?"])

t.format(country="France")

'Human: What is the capital of France?'

In [18]:
examples = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

In [7]:
chat.predict("what do know about France?")

France is a country located in Western Europe. It is known for its rich history, culture, and contributions to art, literature, and philosophy. Here are some key points about France:

1. Capital: The capital city of France is Paris, which is also one of the most visited cities in the world.

2. Language: The official language is French, and it is spoken by the majority of the population. French is also one of the most widely taught second languages globally.

3. History: France has a long and influential history, including the French Revolution in the late 18th century, which led to the establishment of the First French Republic. It has also been involved in major conflicts like World War I and World War II.

4. Culture: France is renowned for its cultural contributions, including art, fashion, cuisine, and literature. It is home to famous artists like Claude Monet, Edgar Degas, and Auguste Rodin. French cuisine is highly regarded worldwide, and French fashion is synonymous with elegan

'France is a country located in Western Europe. It is known for its rich history, culture, and contributions to art, literature, and philosophy. Here are some key points about France:\n\n1. Capital: The capital city of France is Paris, which is also one of the most visited cities in the world.\n\n2. Language: The official language is French, and it is spoken by the majority of the population. French is also one of the most widely taught second languages globally.\n\n3. History: France has a long and influential history, including the French Revolution in the late 18th century, which led to the establishment of the First French Republic. It has also been involved in major conflicts like World War I and World War II.\n\n4. Culture: France is renowned for its cultural contributions, including art, fashion, cuisine, and literature. It is home to famous artists like Claude Monet, Edgar Degas, and Auguste Rodin. French cuisine is highly regarded worldwide, and French fashion is synonymous wi

In [21]:
example_template = """
    Human: {question}
    AI: {answer}
"""

# example_prompt = PromptTemplate.from_template(example_template)

example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

In [22]:
prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"]
)

prompt.format(country="Germany")

'Human: What do you know about France?\nAI: \nHere is what I know:\nCapital: Paris\nLanguage: French\nFood: Wine and Cheese\nCurrency: Euro\n\n\nHuman: What do you know about Italy?\nAI: \nI know this:\nCapital: Rome\nLanguage: Italian\nFood: Pizza and Pasta\nCurrency: Euro\n\n\nHuman: What do you know about Greece?\nAI: \nI know this:\nCapital: Athens\nLanguage: Greek\nFood: Souvlaki and Feta Cheese\nCurrency: Euro\n\n\nHuman: What do you know about Germany?'

In [23]:
chain = prompt | chat

chain.invoke({
    "country": "Germany"
})

AI: 
I know this:
Capital: Berlin
Language: German
Food: Bratwurst and Sauerkraut
Currency: Euro

AIMessageChunk(content='AI: \nI know this:\nCapital: Berlin\nLanguage: German\nFood: Bratwurst and Sauerkraut\nCurrency: Euro')

In [24]:
chain.invoke({
    "country": "Turkey"
})

AI: 
I know this:
Capital: Ankara
Language: Turkish
Food: Kebab and Baklava
Currency: Turkish Lira

AIMessageChunk(content='AI: \nI know this:\nCapital: Ankara\nLanguage: Turkish\nFood: Kebab and Baklava\nCurrency: Turkish Lira')

# 4.2 FewShotChatMessagePromptTemplate

In [26]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate, FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

from dotenv import load_dotenv

load_dotenv()

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [28]:
example_prompt = ChatPromptTemplate.from_messages([
    ("human", "What do you know about {country}?"),
    ("ai", "{answer}")
])

In [32]:
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert."),
    example_prompt,
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat

chain.invoke({
    "country": "Germany"
})

I know this:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro
        Germany is known for its rich history, beautiful landscapes, and technological advancements. It is also famous for its beer culture and Oktoberfest, a traditional festival held in Munich. Germany is located in central Europe and is the largest economy in the European Union.

AIMessageChunk(content='I know this:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro\n        Germany is known for its rich history, beautiful landscapes, and technological advancements. It is also famous for its beer culture and Oktoberfest, a traditional festival held in Munich. Germany is located in central Europe and is the largest economy in the European Union.')

In [33]:
example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert, you give short answers."),
    example_prompt,
    ("human", "What do you know about {country}?")
])

chain = final_prompt | chat

chain.invoke({
    "country": "Germany"
})


        I know this:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro
        

AIMessageChunk(content='\n        I know this:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro\n        ')

In [34]:
chain.invoke({
    "country": "Thailand"
})


        I know this:
        Capital: Bangkok
        Language: Thai
        Food: Pad Thai and Tom Yum Soup
        Currency: Thai Baht
        

AIMessageChunk(content='\n        I know this:\n        Capital: Bangkok\n        Language: Thai\n        Food: Pad Thai and Tom Yum Soup\n        Currency: Thai Baht\n        ')

# 4.3 LengthBasedExampleSelector

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector

from dotenv import load_dotenv

load_dotenv()

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

In [4]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [10]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = LengthBasedExampleSelector(
    examples = examples,
    example_prompt=example_prompt,
    max_length=180,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

In [11]:
prompt.format(country="Brazil")

'Human: What do you know about France?\nAI: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Italy?\nAI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [12]:
chain = prompt | chat

chain.invoke({
    "country": "Germany"
})

AI: 
        I know the following about Germany:
        Capital: Berlin
        Language: German
        Food: Bratwurst and Sauerkraut
        Currency: Euro

AIMessageChunk(content='AI: \n        I know the following about Germany:\n        Capital: Berlin\n        Language: German\n        Food: Bratwurst and Sauerkraut\n        Currency: Euro')

# 4.3 LengthBasedExampleSelector

## RandomExampleSelector

In [13]:
from typing import Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import example_selector
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.example_selector.base import BaseExampleSelector

from dotenv import load_dotenv
load_dotenv()

# class RandomExampleSelector(BaseExampleSelector):

#     def select_examples(self, input_variables: Dict[str, str]) -> List[dict]:
#         return super().select_examples(input_variables)

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

In [14]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [16]:
class RandomExampleSelector(BaseExampleSelector):

    def __init__(self, examples):
        self.examples = examples

    def add_example(self, example):
        self.examples.append(example)

    def select_examples(self, input_variables):
        from random import choice
        
        return [choice(self.examples)]

In [17]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

example_selector = RandomExampleSelector(
    examples = examples,
)

prompt = FewShotPromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

In [18]:
prompt.format(country="Brazil")

'Human: What do you know about Italy?\nAI: \n        I know this:\n        Capital: Rome\n        Language: Italian\n        Food: Pizza and Pasta\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

In [20]:
prompt.format(country="Brazil")

'Human: What do you know about France?\nAI: \n        Here is what I know:\n        Capital: Paris\n        Language: French\n        Food: Wine and Cheese\n        Currency: Euro\n        \n\nHuman: What do you know about Brazil?'

# 4.4 Serialization and Composition

In [1]:
from typing import Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import load_prompt

from dotenv import load_dotenv
load_dotenv()

prompt = load_prompt("./prompt.json")
prompt.format(country="Germany")

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

prompt.format(country="Germany")

'What is the capital of Germany'

In [3]:
prompt = load_prompt("./prompt.yaml")
prompt.format(country="Germany")

'What is the capital of Germany?'

In [3]:
from typing import Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.pipeline import PipelinePromptTemplate

from dotenv import load_dotenv
load_dotenv()

chat = ChatOpenAI(
    temperature=0.1, streaming=True, callbacks=[StreamingStdOutCallbackHandler()]
)

In [4]:
intro = PromptTemplate.from_template(
    """
    You are a role playing assistant.
    And you are impersonating a {character}
"""
)

example = PromptTemplate.from_template(
    """
    This is an example of how you talk:

    Human: {example_question}
    You: {example_answer}
"""
)

start = PromptTemplate.from_template(
    """
    Start now!

    Human: {question}
    You:
"""
)

final = PromptTemplate.from_template(
    """
    {intro}
                                     
    {example}
                              
    {start}
"""
)

In [6]:
full_prompt = PipelinePromptTemplate(
    final_prompt=final,
    pipeline_prompts=prompts,
)

In [12]:
full_prompt.format(
    character="Pirate",
    example_question="What is your location.",
    example_answer="Arrrg! That is a screte!! Arg art",
    question="What is your fav food",
)

'\n    \n    You are a role playing assistant.\n    And you are impersonating a Pirate\n\n                                     \n    \n    This is an example of how you talk:\n\n    Human: What is your location.\n    You: Arrrg! That is a screte!! Arg art\n\n                              \n    \n    Start now!\n\n    Human: What is your fav food\n    You:\n\n'

In [14]:
chain = full_prompt | chat

chain.invoke({
    "character":"Pirate",
    "example_question":"What is your location.",
    "example_answer":"Arrrg! That is a screte!! Arg art",
    "question":"What is your fav food",
})

Arrrg! Me favorite grub be a good ol' plate o' fish 'n' chips! The salty sea air makes me crave it even more, arrr!

AIMessageChunk(content="Arrrg! Me favorite grub be a good ol' plate o' fish 'n' chips! The salty sea air makes me crave it even more, arrr!")

# 4.5 Caching

In [5]:
from typing import Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache

set_llm_cache(InMemoryCache())
set_debug(True)

from dotenv import load_dotenv

load_dotenv()

chat = ChatOpenAI(
    temperature=0.1, 
    # streaming=True, 
    # callbacks=[StreamingStdOutCallbackHandler()]
)

In [2]:
chat.predict("How do you make italian pasta")

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, one tablespoon at a time, until it reaches the desired consistency.\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This allows the gluten to relax and makes the dough easier to roll out.\n6. After resting, divide the dough into smaller portions. Take one portion and flatten it wit

In [3]:
chat.predict("How do you make italian pasta") # save cache의 영향 인퍼 속도를 보면 알 수 있다. 

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork or your fingers, gradually mix the eggs and salt into the flour, incorporating a little at a time.\n4. Once the dough starts to come together, knead it with your hands until it forms a smooth and elastic ball. If the dough is too dry, you can add a little water, one tablespoon at a time, until it reaches the desired consistency.\n5. Once the dough is formed, cover it with a clean kitchen towel or plastic wrap and let it rest for about 30 minutes. This allows the gluten to relax and makes the dough easier to roll out.\n6. After resting, divide the dough into smaller portions. Take one portion and flatten it wit

In [6]:
chat.predict("How do you make pasta") 

[llm/start] [1:llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: How do you make pasta"
  ]
}
[llm/end] [1:llm:ChatOpenAI] [9.79s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "To make pasta, follow these steps:\n\n1. Boil water: Fill a large pot with water and bring it to a rolling boil. Add salt to the water for flavor.\n\n2. Add pasta: Once the water is boiling, carefully add the pasta to the pot. Use about 2 ounces (56 grams) of pasta per person for a main course or 1 ounce (28 grams) for a side dish.\n\n3. Cook pasta: Stir the pasta occasionally to prevent it from sticking together. Cook the pasta according to the package instructions, usually for about 8-12 minutes. Taste a piece of pasta to check if it's cooked to your desired level of tenderness.\n\n4. Drain pasta: Once the pasta is cooked, turn off the heat and carefully pour the contents of the pot into a colander or strainer to drain the water. Shake the colander gen

"To make pasta, follow these steps:\n\n1. Boil water: Fill a large pot with water and bring it to a rolling boil. Add salt to the water for flavor.\n\n2. Add pasta: Once the water is boiling, carefully add the pasta to the pot. Use about 2 ounces (56 grams) of pasta per person for a main course or 1 ounce (28 grams) for a side dish.\n\n3. Cook pasta: Stir the pasta occasionally to prevent it from sticking together. Cook the pasta according to the package instructions, usually for about 8-12 minutes. Taste a piece of pasta to check if it's cooked to your desired level of tenderness.\n\n4. Drain pasta: Once the pasta is cooked, turn off the heat and carefully pour the contents of the pot into a colander or strainer to drain the water. Shake the colander gently to remove excess water.\n\n5. Serve: Transfer the drained pasta to a serving dish or individual plates. You can serve it plain or toss it with your favorite sauce, such as marinara, Alfredo, or pesto. Add any desired toppings or ga

In [1]:
from typing import Dict, List
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.globals import set_llm_cache, set_debug
from langchain.cache import InMemoryCache, SQLiteCache

set_llm_cache(SQLiteCache("cache.db"))


from dotenv import load_dotenv

load_dotenv()

chat = ChatOpenAI(
    temperature=0.1, 
    # streaming=True, 
    # callbacks=[StreamingStdOutCallbackHandler()]
)

In [2]:
chat.predict("How do you make italian pasta")

"To make Italian pasta, you will need the following ingredients:\n\n- 2 cups of all-purpose flour\n- 2 large eggs\n- 1/2 teaspoon of salt\n- Water (if needed)\n\nHere's a step-by-step guide to making Italian pasta:\n\n1. On a clean surface or in a large mixing bowl, pour the flour and create a well in the center.\n2. Crack the eggs into the well and add the salt.\n3. Using a fork, beat the eggs and gradually incorporate the flour from the sides of the well. Continue mixing until a dough starts to form.\n4. Once the dough becomes too stiff to mix with a fork, use your hands to knead it. Knead for about 5-10 minutes until the dough is smooth and elastic. If the dough is too dry, add a little water, one tablespoon at a time, until it reaches the desired consistency.\n5. Shape the dough into a ball and cover it with a clean kitchen towel or plastic wrap. Let it rest for at least 30 minutes to allow the gluten to relax.\n6. After resting, divide the dough into smaller portions for easier ha

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from dotenv import load_dotenv

load_dotenv()

chat = ChatOpenAI(
    temperature=0.1, 
    # streaming=True, 
    # callbacks=[StreamingStdOutCallbackHandler()]
)

with get_openai_callback() as usage:
    chat.predict("What is the recipie for soju")
    print(usage)

Tokens Used: 303
	Prompt Tokens: 15
	Completion Tokens: 288
Successful Requests: 1
Total Cost (USD): $0.0005985


In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.callbacks import get_openai_callback
from dotenv import load_dotenv

load_dotenv()

chat = ChatOpenAI(
    temperature=0.1, 
    # streaming=True, 
    # callbacks=[StreamingStdOutCallbackHandler()]
)

with get_openai_callback() as usage:
    a = chat.predict("What is the recipie for soju")
    b = chat.predict("What is the recipie for bread")
    print(a, b, "\n")
    print(usage)
    # print(usage.total_cost)
    # print(usage.total_tokens)
    # print(usage.completion_tokens)

Here is a simple recipe for making soju:

Ingredients:
- 1.5 liters of water
- 1 cup of rice
- 1 tablespoon of yeast
- 1 cup of sugar

Instructions:
1. Rinse the rice thoroughly until the water runs clear.
2. In a large pot, add the rinsed rice and water. Bring it to a boil and then reduce the heat to low. Let it simmer for about 30 minutes or until the rice is fully cooked and soft.
3. Remove the pot from heat and let it cool down to room temperature.
4. Once the rice has cooled, transfer it to a large container or jar.
5. Dissolve the yeast in a small amount of warm water and add it to the rice mixture.
6. Stir in the sugar until it is fully dissolved.
7. Cover the container or jar with a clean cloth or plastic wrap and let it ferment for about a week in a cool, dark place. Stir the mixture once a day.
8. After a week, strain the mixture through a cheesecloth or fine mesh strainer to remove any solids.
9. Transfer the strained liquid to a clean bottle or bottles and refrigerate befor

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

chat = OpenAI(
    temperature=0.1, 
    max_tokens=450,
    model="gpt-3.5-turbo-16k"
    # streaming=True, 
    # callbacks=[StreamingStdOutCallbackHandler()]
)

chat.save("model.json")

In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.llms.openai import OpenAI
from langchain.llms.loading import load_llm
from dotenv import load_dotenv

load_dotenv()

chat = load_llm("model.json")

chat

/home/aeolian83/anaconda3/envs/clone_gpt_plugin/lib/python3.11/site-packages/langchain/llms/openai.py:216: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/home/aeolian83/anaconda3/envs/clone_gpt_plugin/lib/python3.11/site-packages/langchain/llms/openai.py:811: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


OpenAIChat(client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo-16k', model_kwargs={'temperature': 0.1, 'max_tokens': 450, 'top_p': 1, 'frequency_penalty': 0, 'presence_penalty': 0, 'n': 1, 'request_timeout': None, 'logit_bias': {}})